In [ ]:
!conda install matplotlib-venn
!conda install pandas numpy matplotlib seaborn scikit-learn matplotlib-venn pathlib

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/atharva/miniforge/envs/rna_env

  added / updated specs:
    - matplotlib-venn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.1.0               |       hb03c661_4          19 KB  conda-forge
    brotli-bin-1.1.0           |       hb03c661_4          19 KB  conda-forge
    contourpy-1.3.3            |  py313h7037e92_2         290 KB  conda-forge
    cycler-0.12.1              |     pyhd8ed1ab_1          13 KB  conda-forge
    fonttools-4.59.2           |  py313h3dea7bd_0         2.8 MB  conda-forge
    kiwisolver-1.4.9           |  py313hc8edb43_1          75 

In [1]:
!STAR --version

2.7.11b


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import subprocess
from pathlib import Path
from matplotlib_venn import venn3
from functools import reduce
import shutil
import sys
import os, subprocess, shlex
# Parallel Processing with Multiprocessing
import gzip
from multiprocessing import Pool, cpu_count
# from Bio import SeqIO
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import shutil
import subprocess
from pathlib import Path
import multiprocessing as mp
import glob, math
import shlex

In [3]:
p1 = "/mnt/a/Projects/RA_Assignment/Patient1"

# Patient1
if not any(os.scandir(p1)):  # empty dir
    print("Patient1 FASTQs not found → downloading...")
    %cd $p1
    !bash /content/drive/MyDrive/Bio_Informatics_RA_Assignment/ena-file-download-read_run-SAMN03431245-fastq_ftp-20250830-1946.sh

else:
    print("Patient1 FASTQs already exist → skipping download.")

Patient1 FASTQs already exist → skipping download.


In [4]:
def print_fastq_head(file_path, num_reads=5):
  if( file_path.endswith('.gz')):
    open_func = gzip.open
    mode = 'rt'
  else:
    open_func = open
    mode = 'r'
  with open_func(file_path, mode) as f:
    for i in range(num_reads * 4):
      line = f.readline()
      if not line:
        break
      print(line.rstrip())

import gzip

def get_fastq_read_count(file_path):
    """
    Counts the total number of reads in a FASTQ file.
    Handles both gzipped (.gz) and uncompressed files.
    """
    if file_path.endswith('.gz'):
        open_func = gzip.open
        mode = 'rt'
    else:
        open_func = open
        mode = 'r'

    total_reads = 0
    with open_func(file_path, mode) as f:
        for i, line in enumerate(f):
            if (i + 1) % 4 == 0:
                total_reads += 1
    
    return total_reads

In [5]:
print_fastq_head('/mnt/a/Projects/RA_Assignment/Patient1/SRR1918235_1.fastq.gz',2)
# total_reads = get_fastq_read_count('A:\\Projects\\RA_Assignment\\Patient1\\SRR1918235_1.fastq.gz')
# print(f"Total Number of reads: {total_reads}")

@SRR1918235.1 HWI-ST766:119:C4N1AACXX:3:1114:4077:54247/1
CCGAAATCTGTGCAGAGGAGAACGCAGCTCCGCCCTCGCGGTGCTCTCCGGGTCTGTGCTGAGGAGAACGCAACTCCGCCGAGATCGGAAGAGCACACGT
+
BBBFBFFFBBBFBBFBBF<FFFBBBBFFBBBF<<FFFFB'0<BBB0BFFB007<BB<B<<<<B7BB<B<BBBB7B0'0'07<B<BBBB0B7707BB0B##
@SRR1918235.2 HWI-ST766:119:C4N1AACXX:3:2214:9166:51017/1
CCCACCAGCAATGTCTAGGAATGCCTGTTTCTCCACAAAGTGTTTACTTTTGGATTTTTGCCAGTCTAACAGGTGAAGCCCTGGAAATTCTTATTAGTGA
+
BBBFFFFFFFFBFFFFFIIFFFFFIFFFFFFFBFFFBFIFBFFFFFIIFFIFFIIIIFFIBFIIFFFIBFIBFBFBB7B<BFBBFBBFBBFFFFFFFBB<


In [6]:
import shutil, sys, os, subprocess

print("Python executable:", sys.executable)
print("Kernel PATH:", os.environ.get("PATH"))
print("shutil.which('fastqc') ->", shutil.which("fastqc"))
print("shutil.which('STAR') ->", shutil.which("STAR"))
print("shutil.which('samtools') ->", shutil.which("samtools"))
print("shutil.which('featureCounts') ->", shutil.which("featureCounts"))

# Try running fastqc --version (capture output)
try:
    out = subprocess.run(["fastqc","--version"], capture_output=True, text=True, check=True)
    print("fastqc --version:", out.stdout.strip() or out.stderr.strip())
except Exception as e:
    print("Error running fastqc --version:", repr(e))


Python executable: /home/atharva/miniforge/envs/rna_env/bin/python
Kernel PATH: /home/atharva/miniforge/envs/rna_env/bin:/home/atharva/miniforge/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Files/Eclipse Adoptium/jdk-17.0.16.8-hotspot/bin:/mnt/c/Program Files (x86)/Common Files/Intel/Shared Libraries/redist/intel64/compiler:/mnt/c/Program Files/Common Files/Oracle/Java/javapath:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0:/mnt/c/Windows/System32/OpenSSH:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Program Files/NVIDIA Corporation/NVIDIA App/NvDLISR:/mnt/c/Program Files/Git/cmd:/mnt/c/Users/athar/AppData/Local/Programs/Python/Python311/Lib/site-packages/pip:/mnt/c/Users/athar/AppData/Local/Programs/Python/Python311/Scripts:/mnt/c/Users/athar/AppData/Roaming/nvm:/mnt/c/Program Files/nodejs:/mnt/c/Program Files/Micr

In [20]:
import os
import subprocess
import shlex
import time
from pathlib import Path
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import tempfile
import shutil

class RNASeqPipeline:
    """RNA-seq analysis pipeline for gene expression data (not TCR/BCR)"""
    
    def __init__(self, star_path="STAR", samtools_path="samtools", 
                 featurecounts_path="featureCounts", total_threads=None):
        self.star_path = star_path
        self.samtools_path = samtools_path
        self.featurecounts_path = featurecounts_path
        self.total_threads = total_threads or os.cpu_count() or 4
        
    def run_command(self, cmd, cwd=None, timeout=None, env=None):
        """
        Run a command safely.
        - cmd: list (recommended) or string
        - returns subprocess.CompletedProcess on success
        - raises CalledProcessError on failure
        """
        import subprocess, shlex, time
        print("\n$ " + (shlex.join(cmd) if isinstance(cmd, (list,tuple)) else str(cmd)))
        try:
            if isinstance(cmd, (list, tuple)):
                # run directly without shell (safer)
                result = subprocess.run(cmd, cwd=cwd, check=True, capture_output=True, text=True, timeout=timeout, env=env)
            else:
                # cmd is a string; run via shell
                result = subprocess.run(cmd, cwd=cwd, check=True, capture_output=True, text=True, timeout=timeout, shell=True, env=env)
            if result.stdout:
                print(result.stdout)
            if result.stderr:
                # print warnings but not fail
                print(result.stderr)
            return result
        except subprocess.CalledProcessError as e:
            print(f"Command failed (returncode={e.returncode})")
            if e.stdout:
                print("STDOUT:", e.stdout)
            if e.stderr:
                print("STDERR:", e.stderr)
            raise
        except subprocess.TimeoutExpired as e:
            print(f"Command timed out after {timeout} seconds")
            raise

    
    def check_tools_installed(self):
        """Check if required CLI tools are available in PATH (kernel env)."""
        import shutil, subprocess
        tools = [
            ("STAR", "STAR aligner"),
            ("samtools", "SAMtools"),
            ("featureCounts", "featureCounts (subread)"),
            ("fastqc", "FastQC"),
            ("seqkit", "SeqKit")
        ]
        missing = []
        for exe, nice in tools:
            path = shutil.which(exe)
            if path:
                # try quick version check when possible
                try:
                    subprocess.run([exe, "--version"], capture_output=True, check=False, timeout=10)
                    print(f"✓ {nice} found: {path}")
                except Exception:
                    print(f"✓ {nice} found: {path} (version probe failed but binary exists)")
            else:
                print(f"✗ {nice} ({exe}) not found in PATH")
                missing.append(f"{nice} ({exe})")
        if missing:
            print("\nMissing tools:", ", ".join(missing))
            print("If they are installed in a conda env, start Jupyter from that env or add their bin directory to PATH.")
            return False
        return True

    
    def quality_control(self, fastq1, fastq2, output_dir, threads=None):
        """Run FastQC in non-interactive CLI mode and produce HTML outputs in output_dir."""
        from pathlib import Path
        import shutil
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        threads = threads or min(4, self.total_threads)
        fastqc_exe = shutil.which("fastqc")
        if not fastqc_exe:
            raise FileNotFoundError("FastQC not found in PATH (check environment).")
        print("Running FastQC (non-interactive)...")
        cmd = [fastqc_exe, "-o", str(output_dir), "-t", str(threads), str(fastq1), str(fastq2)]
        self.run_command(cmd, timeout=1800)
        # locate generated HTML (fastqc names output like <sample>_fastqc.html)
        # Optionally return the paths so the caller can embed them
        htmls = list(output_dir.glob("*_fastqc.html"))
        zips = list(output_dir.glob("*_fastqc.zip"))
        print(f"FastQC produced {len(htmls)} HTML report(s) and {len(zips)} zip(s)")
        return {"htmls": [str(p) for p in htmls], "zips": [str(p) for p in zips]}

    
    def basic_read_stats(self, fastq1, fastq2):
        """Get basic statistics about the reads"""
        print("Analyzing read statistics...")
        
        # Count reads
        result1 = subprocess.run(
            ["seqkit", "stats", str(fastq1)], 
            capture_output=True, text=True, shell=True
        )
        result2 = subprocess.run(
            ["seqkit", "stats", str(fastq2)], 
            capture_output=True, text=True, shell=True
        )
        
        print("Read 1 stats:")
        print(result1.stdout)
        print("Read 2 stats:")
        print(result2.stdout)
        
        return result1.stdout, result2.stdout
    
    def download_reference_genome(self, output_dir, species="human", retries=3, chunk_size=1024*1024):
        """
        Download reference genome and GTF using Python (no wget).
        Streams to disk and avoids shell/wget issues.
    
        Returns: (genome_fa_path, gtf_unzipped_path)
        """
        from pathlib import Path
        import urllib.request
        import shutil
        import gzip
        import time
        import sys
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
    
        if species.lower() != "human":
            raise ValueError("Only human reference genome is implemented")
    
        # URLs (Gencode release 44 as in original)
        genome_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/GRCh38.primary_assembly.genome.fa.gz"
        gtf_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.primary_assembly.annotation.gtf.gz"
    
        genome_gz = output_dir / "GRCh38.primary_assembly.genome.fa.gz"
        gtf_gz = output_dir / "gencode.v44.primary_assembly.annotation.gtf.gz"
    
        def _download(url, target, retries=retries):
            attempt = 0
            while attempt < retries:
                try:
                    attempt += 1
                    print(f"Downloading (attempt {attempt}): {url}")
                    req = urllib.request.urlopen(url, timeout=60)
                    total = req.getheader('Content-Length')
                    if total:
                        total = int(total)
                        print(f"Total bytes: {total:,}")
                    with open(target, "wb") as out:
                        downloaded = 0
                        while True:
                            chunk = req.read(chunk_size)
                            if not chunk:
                                break
                            out.write(chunk)
                            downloaded += len(chunk)
                            if total:
                                pct = downloaded/total*100
                                sys.stdout.write(f"\r{downloaded:,}/{total:,} bytes ({pct:.1f}%)")
                            else:
                                sys.stdout.write(f"\r{downloaded:,} bytes")
                            sys.stdout.flush()
                    print("\nDownload finished:", target)
                    return True
                except Exception as e:
                    print(f"\nDownload attempt {attempt} failed: {e}")
                    time.sleep(3 * attempt)
            raise RuntimeError(f"Failed to download {url} after {retries} attempts")
    
        # download files only if not already present
        if not genome_gz.exists():
            _download(genome_url, genome_gz)
        else:
            print("Genome gz already exists, skipping download:", genome_gz)
    
        if not gtf_gz.exists():
            _download(gtf_url, gtf_gz)
        else:
            print("GTF gz already exists, skipping download:", gtf_gz)
    
        # Decompress gz -> .fa and .gtf (if not already exist)
        genome_fa = output_dir / "GRCh38.primary_assembly.genome.fa"
        gtf_unzipped = output_dir / "gencode.v44.primary_assembly.annotation.gtf"
    
        if not genome_fa.exists():
            print("Decompressing genome fasta (this will keep .gz file).")
            with gzip.open(genome_gz, "rb") as f_in, open(genome_fa, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        else:
            print("Genome fasta already exists:", genome_fa)
    
        if not gtf_unzipped.exists():
            print("Decompressing GTF.")
            with gzip.open(gtf_gz, "rb") as f_in, open(gtf_unzipped, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        else:
            print("GTF already exists:", gtf_unzipped)
    
        return str(genome_fa), str(gtf_unzipped)

    
    def build_star_index(self, genome_fasta, gtf_file, index_dir, max_threads=None):
        """
        Robust STAR index builder with fallbacks for memory-limited environments.
        Tries:
          1) normal build with capped threads
          2) retry with --limitGenomeGenerateRAM and fewer threads
          3) build on canonical chromosomes (chr1..22,X,Y,MT)
        Returns path to index_dir used.
        """
        from pathlib import Path
        import shutil, subprocess, sys, os
        index_dir = Path(index_dir)
        index_dir.mkdir(parents=True, exist_ok=True)
        max_threads = int(max_threads or min(self.total_threads, 8))  # cap default threads to 8 for safety
    
        # small helper to call STAR and capture return
        def _run_star(cmd, timeout=7200):
            try:
                print("Running:", " ".join(cmd))
                subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=timeout)
                return 0, None
            except subprocess.CalledProcessError as e:
                rc = e.returncode
                stderr = e.stderr or e.stdout
                print(f"STAR failed (rc={rc}). stderr snippet:\n", (stderr or "")[:1000])
                return rc, stderr
            except Exception as e:
                print("Exception running STAR:", e)
                return -1, str(e)
    
        # If index already exists skip
        if (index_dir / "genomeParameters.txt").exists():
            print("STAR index already exists at", index_dir)
            return str(index_dir)
    
        print("Attempt 1: build STAR index with threads =", max_threads)
        cmd1 = [
            self.star_path, "--runMode", "genomeGenerate",
            "--genomeDir", str(index_dir),
            "--genomeFastaFiles", str(genome_fasta),
            "--sjdbGTFfile", str(gtf_file),
            "--sjdbOverhang", "100",
            "--runThreadN", str(max_threads)
        ]
        rc, stderr = _run_star(cmd1)
        if rc == 0:
            print("STAR index built successfully (attempt 1).")
            return str(index_dir)
    
        # If rc 137 or similar, try lowering memory/threads
        print("\nAttempt 2: retry with lowered threads and limitGenomeGenerateRAM")
        lower_threads = max(4, int(max_threads/2))
        # try with 40GB
        limit_ram = 40_000_000_000
        cmd2 = cmd1 + ["--runThreadN", str(lower_threads), "--limitGenomeGenerateRAM", str(limit_ram)]
        rc2, stderr2 = _run_star(cmd2)
        if rc2 == 0:
            print("STAR index built successfully (attempt 2).")
            return str(index_dir)
    
        # Attempt 3: build index on canonical chromosomes only (smaller FASTA/GTF)
        print("\nAttempt 3: build index on canonical chromosomes (chr1..22,X,Y,MT).")
        # prepare reduced fasta and gtf
        work = index_dir.parent / "reduced_reference"
        work.mkdir(parents=True, exist_ok=True)
        reduced_fa = work / (Path(genome_fasta).stem + ".canonical.fa")
        reduced_gtf = work / (Path(gtf_file).stem + ".canonical.gtf")
    
        # create canonical chromosome list - adjust names if your FASTA uses no 'chr' prefix
        canonical = ["chr"+str(i) for i in range(1,23)] + ["chrX","chrY","chrM"]
        # try both with and without 'chr' if not found
        print("Creating canonical FASTA/GTF; this may take a moment.")
        try:
            # attempt extraction using samtools faidx if available
            if shutil.which("samtools"):
                idx = str(genome_fasta) + ".fai"
                if not Path(idx).exists():
                    print("Indexing FASTA with samtools faidx...")
                    subprocess.run(["samtools", "faidx", str(genome_fasta)], check=True)
                # determine actual contig names in FASTA
                contigs = []
                with open(str(genome_fasta)+".fai") as f:
                    contigs = [l.split('\t',1)[0] for l in f]
                # intersect canonical with actual contigs (try both styles)
                use_contigs = [c for c in canonical if c in contigs]
                if not use_contigs:
                    # try without 'chr' prefix
                    canonical2 = [c.replace("chr","") for c in canonical]
                    use_contigs = [c for c in canonical2 if c in contigs]
                if not use_contigs:
                    print("Could not find canonical contigs in FASTA index; aborting reduced-reference approach.")
                else:
                    # write a temporary list file and use samtools faidx to extract
                    listf = work / "canonical_list.txt"
                    listf.write_text("\n".join(use_contigs))
                    print("Extracting contigs:", use_contigs[:10], "... (total {})".format(len(use_contigs)))
                    subprocess.run(["samtools", "faidx", str(genome_fasta), "-r", str(listf), "-o", str(reduced_fa)], check=True)
                    # filter GTF by contigs
                    with open(reduced_gtf, "w") as out:
                        with open(gtf_file) as g:
                            for line in g:
                                if line.startswith("#"):
                                    out.write(line)
                                    continue
                                parts = line.split("\t")
                                if parts[0] in use_contigs:
                                    out.write(line)
            else:
                print("samtools not found; cannot auto-extract canonical contigs. Please create reduced FASTA/GTf manually.")
        except Exception as e:
            print("Error creating reduced reference:", e)
    
        # if reduced files created, run STAR on them
        if reduced_fa.exists() and reduced_gtf.exists():
            reduced_index = work / "star_index_canonical"
            reduced_index.mkdir(parents=True, exist_ok=True)
            cmd3 = [
                self.star_path, "--runMode", "genomeGenerate",
                "--genomeDir", str(reduced_index),
                "--genomeFastaFiles", str(reduced_fa),
                "--sjdbGTFfile", str(reduced_gtf),
                "--sjdbOverhang", "100",
                "--runThreadN", "4",
                "--limitGenomeGenerateRAM", str(30_000_000_000)  # 30GB
            ]
            rc3, stderr3 = _run_star(cmd3)
            if rc3 == 0:
                print("STAR index built successfully on reduced reference:", reduced_index)
                return str(reduced_index)
            else:
                print("Reduced-reference STAR build also failed (rc={}).".format(rc3))
        else:
            print("Reduced FASTA/GTF were not created; cannot attempt reduced index build.")
    
        # If we reach here, all attempts failed
        raise RuntimeError("STAR genomeGenerate failed after multiple attempts. "
                           "Consider running on a machine with more RAM (>=64GB), "
                           "or using a prebuilt STAR index.")

    
    def align_reads(self, fastq1, fastq2, star_index, output_prefix, sample_name):
        """Align reads using STAR"""
        
        print(f"Aligning reads for {sample_name}...")
        
        star_cmd = [
            self.star_path,
            "--genomeDir", star_index,
            "--readFilesIn", str(fastq1), str(fastq2),
            "--readFilesCommand", "zcat",  # For .gz files
            "--outFileNamePrefix", f"{output_prefix}_",
            "--outSAMtype", "BAM", "SortedByCoordinate",
            "--outSAMunmapped", "Within",
            "--quantMode", "GeneCounts",  # Get gene counts directly
            "--runThreadN", str(self.total_threads)
        ]
        
        self.run_command(star_cmd, timeout=3600)
        
        # Output files
        bam_file = f"{output_prefix}_Aligned.sortedByCoord.out.bam"
        counts_file = f"{output_prefix}_ReadsPerGene.out.tab"
        log_file = f"{output_prefix}_Log.final.out"
        
        return bam_file, counts_file, log_file
    
    def index_bam(self, bam_file):
        """Index BAM file"""
        print(f"Indexing BAM file: {bam_file}")
        
        index_cmd = [self.samtools_path, "index", bam_file]
        self.run_command(index_cmd, timeout=600)
        
        return f"{bam_file}.bai"

def analyze_aml_rnaseq(fastq1, fastq2, output_dir, sample_name, 
                      reference_dir=None, skip_reference=False):
    """
    Complete RNA-seq analysis pipeline for AML data
    """
    
    pipeline = RNASeqPipeline()
    
    # Check tools
    if not pipeline.check_tools_installed():
        print("Please install missing tools before proceeding")
        return None
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    qc_dir = output_dir / "quality_control"
    alignment_dir = output_dir / "alignments"
    reference_dir = Path(reference_dir) if reference_dir else output_dir / "reference"
    
    try:
        # Step 1: Quality control
        print("\n" + "="*50)
        print("STEP 1: Quality Control")
        print("="*50)
        
        pipeline.quality_control(fastq1, fastq2, qc_dir)
        pipeline.basic_read_stats(fastq1, fastq2)
        
        if skip_reference:
            print("\nSkipping reference genome setup (skip_reference=True)")
            print("Make sure you have:")
            print("1. Reference genome FASTA file")
            print("2. Gene annotation GTF file") 
            print("3. Pre-built STAR index")
            return None
        
        # Step 2: Reference genome setup
        print("\n" + "="*50)
        print("STEP 2: Reference Genome Setup")
        print("="*50)
        
        genome_fa, gtf_file = pipeline.download_reference_genome(reference_dir)
        
        # Step 3: Build STAR index
        print("\n" + "="*50)
        print("STEP 3: Building STAR Index")
        print("="*50)
        
        star_index = pipeline.build_star_index(
            genome_fa, gtf_file, reference_dir / "star_index"
        )
        
        # Step 4: Alignment
        print("\n" + "="*50)
        print("STEP 4: Read Alignment")
        print("="*50)
        
        alignment_dir.mkdir(parents=True, exist_ok=True)
        output_prefix = alignment_dir / sample_name
        
        bam_file, counts_file, log_file = pipeline.align_reads(
            fastq1, fastq2, star_index, str(output_prefix), sample_name
        )
        
        # Step 5: Index BAM
        print("\n" + "="*50)
        print("STEP 5: Indexing BAM file")
        print("="*50)
        
        bam_index = pipeline.index_bam(bam_file)
        
        # Step 6: Parse results
        print("\n" + "="*50)
        print("STEP 6: Results Summary")
        print("="*50)
        
        results = {
            "sample_name": sample_name,
            "bam_file": bam_file,
            "bam_index": bam_index,
            "counts_file": counts_file,
            "log_file": log_file,
            "qc_dir": str(qc_dir),
            "reference_genome": genome_fa,
            "annotation": gtf_file,
            "star_index": star_index
        }
        
        # Parse gene counts
        if Path(counts_file).exists():
            counts_df = pd.read_csv(counts_file, sep='\t', header=None, skiprows=4)
            counts_df.columns = ['gene_id', 'unstranded', 'stranded_first', 'stranded_second']
            
            total_genes = len(counts_df)
            expressed_genes = len(counts_df[counts_df['unstranded'] > 0])
            total_reads = counts_df['unstranded'].sum()
            
            print(f"Total genes: {total_genes:,}")
            print(f"Expressed genes: {expressed_genes:,}")
            print(f"Total aligned reads: {total_reads:,}")
            
            results["gene_counts"] = counts_df
            results["total_genes"] = total_genes
            results["expressed_genes"] = expressed_genes
            results["total_reads"] = total_reads
        
        print(f"\nRNA-seq analysis completed successfully!")
        print(f"Results saved in: {output_dir}")
        
        return results
        
    except Exception as e:
        print(f"Pipeline failed: {e}")
        import traceback
        traceback.print_exc()
        raise

In [21]:
# Step 1 Pulling the data systematically from the mounted drive
base_dir = "/mnt/a/Projects/RA_Assignment"

patients = ["Patient1"]

patient_files = {"Patient1": ("SRR1918235_1.fastq.gz", "SRR1918235_2.fastq.gz")}

# Output dir for results
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok=True)

### Added: Visualization helpers

This cell adds plotting and FastQC parsing helpers to `RNASeqPipeline`:
- `parse_star_log`
- `plot_star_metrics`
- `plot_top_expressed_genes`
- `plot_expression_distribution`
- `plot_top_variable_genes_heatmap`
- `parse_fastqc_summary`

These are appended to the end of the notebook and monkeypatched onto the class if present.

In [22]:

# --- ADDED: Visualization & FastQC parsing helpers for RNASeqPipeline ---
import matplotlib.pyplot as plt
from matplotlib import ticker
import math
import numpy as np
import pandas as pd
from pathlib import Path

def parse_star_log(self, file):
    """Parse STAR Log.final.out into a dict of metric -> value.
    STAR log lines look like: '                             Started job on |   Nov 13 15:43:11'
    We split on the '|' character and strip whitespace.
    """
    metrics = {}
    with open(file) as f:
        for line in f:
            line = line.strip()
            if '|' in line:
                parts = line.split('|')
                if len(parts) == 2:
                    key, val = [x.strip() for x in parts]
                    metrics[key] = val
    return metrics

def plot_star_metrics(self, log_file, ax=None, figsize=(8,4), save=None):
    """Create a small bar chart / summary of important STAR metrics.
    Returns the parsed metrics dict and matplotlib figure (ax).
    """
    metrics = self.parse_star_log(log_file)
    
    # pick a standard subset of keys (if present)
    keys_of_interest = [
        'Number of input reads',
        'Average input read length',
        'Uniquely mapped reads number',
        'Uniquely mapped reads %',
        'Number of reads mapped to multiple loci',
        '% of reads mapped to multiple loci',
        'Number of reads mapped to too many loci',
        '% of reads mapped to too many loci',
        'Unmapped reads: too short',
        'Unmapped reads: too many mismatches',
        'Unmapped reads: other'
    ]
    present = [(k, metrics[k]) for k in keys_of_interest if k in metrics]
    if not present:
        # fallback: show a pretty-printed text summary if keys not found
        fig, ax = plt.subplots(figsize=(8,6))
        ax.axis('off')
        txt = '\n'.join(f"{k}: {v}" for k,v in metrics.items())
        ax.text(0, 1, txt, va='top', fontsize=9, family='monospace')
        if save:
            fig.savefig(save, bbox_inches='tight', dpi=150)
        return metrics, ax
    
    labels = [p[0] for p in present]
    vals = []
    for _, v in present:
        # remove trailing % or commas; convert to float if possible
        vv = v.replace('%','').replace(',','').strip()
        try:
            vals.append(float(vv))
        except:
            # keep as NaN so we can skip in plotting numeric bars
            vals.append(math.nan)
    
    # pick numeric entries
    numeric_labels = []
    numeric_vals = []
    for l, val in zip(labels, vals):
        if not math.isnan(val):
            numeric_labels.append(l)
            numeric_vals.append(val)
    
    fig, ax = plt.subplots(figsize=figsize)
    if numeric_vals:
        ax.barh(range(len(numeric_vals)), numeric_vals, align='center')
        ax.set_yticks(range(len(numeric_vals)))
        ax.set_yticklabels(numeric_labels, fontsize=9)
        ax.invert_yaxis()
        ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f"{x:g}"))
        ax.set_xlabel("Value")
        ax.set_title("STAR alignment metrics (selected)")
        plt.tight_layout()
    else:
        ax.axis('off')
        ax.text(0, 1, "No numeric STAR metrics available to plot.", va='top')
    
    if save:
        fig.savefig(save, bbox_inches='tight', dpi=150)
    return metrics, ax

def plot_top_expressed_genes(self, counts_df, top_n=20, ax=None, figsize=(8,6), save=None):
    """
    counts_df expected format: columns like ['gene_id', 'unstranded', 'stranded_first', ...]
    Plot top_n genes by unstranded counts.
    """
    # defensive checks
    if 'unstranded' not in counts_df.columns:
        raise ValueError("counts_df must contain 'unstranded' column produced by STAR (--quantMode GeneCounts)")
    
    top = counts_df.sort_values('unstranded', ascending=False).head(top_n)
    fig, ax = plt.subplots(figsize=figsize)
    ax.barh(range(len(top)), top['unstranded'].values[::-1], align='center')
    ax.set_yticks(range(len(top)))
    ax.set_yticklabels(top['gene_id'].values[::-1], fontsize=9)
    ax.invert_yaxis()
    ax.set_xlabel("Read counts (unstranded)")
    ax.set_title(f"Top {top_n} expressed genes")
    plt.tight_layout()
    
    if save:
        fig.savefig(save, bbox_inches='tight', dpi=150)
    return fig, ax

def plot_expression_distribution(self, counts_df, ax=None, figsize=(6,4), save=None):
    """
    Shows histogram of counts per gene (log10 scale).
    """
    if 'unstranded' not in counts_df.columns:
        raise ValueError("counts_df must contain 'unstranded' column")
    
    counts = counts_df['unstranded'].values
    counts_pos = counts[counts > 0]
    if len(counts_pos) == 0:
        fig, ax = plt.subplots(figsize=figsize)
        ax.text(0.5,0.5,"No positive counts to plot", ha='center')
        return fig, ax
    
    log_counts = np.log10(counts_pos)
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.hist(log_counts, bins=50)
    ax.set_xlabel("log10(read counts)")
    ax.set_ylabel("Number of genes")
    ax.set_title("Distribution of gene expression (log10 counts)")
    plt.tight_layout()
    
    if save:
        fig.savefig(save, bbox_inches='tight', dpi=150)
    return fig, ax

def plot_top_variable_genes_heatmap(self, counts_df, top_n=50, figsize=(8,8), save=None):
    """
    Create a small heatmap of top N most variable genes across columns (if multiple samples).
    If counts_df has only one sample (single column), this will just plot the top N values as a column heatmap.
    """
    # Accept both the STAR counts format and standard gene x sample countframes.
    df = counts_df.copy()
    if 'unstranded' in df.columns and df.shape[1] == 4:
        # STAR output single-sample: convert to gene x 1 sample
        expr = pd.DataFrame(df['unstranded'].values, index=df['gene_id'], columns=['sample'])
    else:
        # assume gene_id is index or first column
        if 'gene_id' in df.columns:
            expr = df.set_index('gene_id').select_dtypes(include=[np.number])
        else:
            expr = df.select_dtypes(include=[np.number])
    
    if expr.shape[1] == 0:
        raise ValueError("No numeric expression columns found to plot heatmap")
    
    # compute variability and pick top N
    var = expr.var(axis=1).sort_values(ascending=False).head(top_n)
    top_expr = expr.loc[var.index]
    
    # simple plotting using imshow
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(np.log10(top_expr + 1), aspect='auto', interpolation='nearest')
    ax.set_yticks(range(len(top_expr.index)))
    ax.set_yticklabels(top_expr.index, fontsize=8)
    ax.set_xticks(range(len(top_expr.columns)))
    ax.set_xticklabels(top_expr.columns, rotation=45, ha='right', fontsize=9)
    ax.set_title(f"Heatmap of top {top_n} variable genes (log10(count+1))")
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("log10(count+1)")
    plt.tight_layout()
    if save:
        fig.savefig(save, bbox_inches='tight', dpi=150)
    return fig, ax

def parse_fastqc_summary(self, fastqc_zip_or_html):
    """Attempt to parse FastQC output (if you produced JSON or can parse HTML).
    This is a light-weight helper that looks for the FastQC summary.txt in a directory or zip.
    Returns a dict of module -> status.
    """
    p = Path(fastqc_zip_or_html)
    summary = {}
    try:
        if p.is_dir():
            summary_file = p / "summary.txt"
            if summary_file.exists():
                for line in open(summary_file):
                    parts = line.strip().split('\t')
                    if len(parts) >= 2:
                        status, module = parts[0], parts[1]
                        summary[module] = status
        else:
            # try to find embedded 'FastQC Report' title in html for a quick status
            txt = p.read_text(encoding='utf-8', errors='ignore')
            if "FastQC Report" in txt:
                summary['report'] = "OK (html detected)"
    except Exception as e:
        summary['error'] = str(e)
    return summary

# Monkeypatch onto RNASeqPipeline class if it exists in the user's environment
try:
    RNASeqPipeline.parse_star_log = parse_star_log
    RNASeqPipeline.plot_star_metrics = plot_star_metrics
    RNASeqPipeline.plot_top_expressed_genes = plot_top_expressed_genes
    RNASeqPipeline.plot_expression_distribution = plot_expression_distribution
    RNASeqPipeline.plot_top_variable_genes_heatmap = plot_top_variable_genes_heatmap
    RNASeqPipeline.parse_fastqc_summary = parse_fastqc_summary
    print("Visualization helpers attached to RNASeqPipeline")
except Exception as e:
    print("Could not attach helpers automatically:", e)


Visualization helpers attached to RNASeqPipeline


In [ ]:
# Usage for your AML data
if __name__ == "__main__":
    
    fastq1 = Path(base_dir) / "Patient1" / "SRR1918235_1.fastq.gz"
    fastq2 = Path(base_dir) / "Patient1" / "SRR1918235_2.fastq.gz"
    output_dir = Path(base_dir) / "rnaseq_results" / "Patient1"
    patient = "Patient1_AML"
    
    print("AML RNA-seq Analysis Pipeline")
    print("="*40)
    print("This dataset contains:")
    print("- Tissue: Bone marrow")
    print("- Cell Type: Acute myeloid leukemia (AML) cells") 
    print("- Data Type: RNA-seq (gene expression)")
    print("- Organism: Homo sapiens")
    print()
    
    print("This pipeline will:")
    print("1. Run quality control (FastQC)")
    print("2. Download human reference genome")
    print("3. Build STAR alignment index")
    print("4. Align reads and count genes")
    print("5. Generate expression analysis")
    print()
    
    # Warning about resource requirements
    print("RESOURCE REQUIREMENTS:")
    print("- Time: 2-4 hours for full pipeline")
    print("- Memory: 30+ GB RAM for STAR index")
    print("- Storage: 50+ GB for reference genome and results")
    print()
    
    proceed = input("Proceed with full analysis? (y/n): ").lower().strip()
    
    if proceed == 'y':
        try:
            results = analyze_aml_rnaseq(
                fastq1=fastq1,
                fastq2=fastq2,
                output_dir=output_dir,
                sample_name=patient
            )
            
            if results:
                print(f"\nAnalysis completed!")
                print(f"Key results:")
                print(f"- BAM file: {results['bam_file']}")
                print(f"- Gene counts: {results['counts_file']}")
                print(f"- Expressed genes: {results.get('expressed_genes', 'N/A')}")
        
        except Exception as e:
            print(f"Analysis failed: {e}")
    
    else:
        print("Analysis cancelled.")
        print("\nTo run just quality control:")
        print("pipeline = RNASeqPipeline()")
        print("pipeline.quality_control(fastq1, fastq2, 'qc_output')")

AML RNA-seq Analysis Pipeline
This dataset contains:
- Tissue: Bone marrow
- Cell Type: Acute myeloid leukemia (AML) cells
- Data Type: RNA-seq (gene expression)
- Organism: Homo sapiens

This pipeline will:
1. Run quality control (FastQC)
2. Download human reference genome
3. Build STAR alignment index
4. Align reads and count genes
5. Generate expression analysis

RESOURCE REQUIREMENTS:
- Time: 2-4 hours for full pipeline
- Memory: 30+ GB RAM for STAR index
- Storage: 50+ GB for reference genome and results



Proceed with full analysis? (y/n):  y


✓ STAR aligner found: /home/atharva/miniforge/envs/rna_env/bin/STAR
✓ SAMtools found: /home/atharva/miniforge/envs/rna_env/bin/samtools
✓ featureCounts (subread) found: /home/atharva/miniforge/envs/rna_env/bin/featureCounts
✓ FastQC found: /home/atharva/miniforge/envs/rna_env/bin/fastqc
✓ SeqKit found: /home/atharva/miniforge/envs/rna_env/bin/seqkit

STEP 1: Quality Control
Running FastQC (non-interactive)...

$ /home/atharva/miniforge/envs/rna_env/bin/fastqc -o /mnt/a/Projects/RA_Assignment/rnaseq_results/Patient1/quality_control -t 4 /mnt/a/Projects/RA_Assignment/Patient1/SRR1918235_1.fastq.gz /mnt/a/Projects/RA_Assignment/Patient1/SRR1918235_2.fastq.gz


In [ ]:
from IPython.display import IFrame, display, HTML
reports = results.get("fastqc_reports")
if reports:
    for html in reports["htmls"]:
        print("Embedding:", html)
        display(IFrame(src=html, width=1000, height=600))
else:
    print("No FastQC HTML reports found in results")
